In [1]:
!pip install evaluate
!pip install rouge-score
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 8.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/sim

In [2]:
import os
import numpy as np
from rouge_score import rouge_scorer
from evaluate import load

In [ ]:
# Only run this if you are interested in calculating BertScore as a metric
# Requirements: 1.5GB+ download
BERT = load("bertscore")

In [15]:
FOLDER = "./results/"
REFERENCE = "Given_Summaries/"
EVAL_INDEX = 480 # The unseen test data index

MODELS = ["TextRank", "GPT2", "DistilGPT", "DistilGPT_filtered", "T5_8epoch", "T5_16epoch", "ChatGPTPrompt"]
METRICS = ["rouge1", "rougeL", "BertScore"]

def load_articles(PATH="", begin_from=0):
    files = sorted([f for f in os.listdir(PATH) if f.endswith('.txt')])

    flist = []
    for file in files[begin_from:]:
        with open(os.path.join(PATH, file), 'r') as f:
            text = f.read()
            flist.append(text)
    
    return flist

In [6]:
def score(model="T5_16epoch", metric="rouge1"):
    reference = load_articles(FOLDER+REFERENCE, EVAL_INDEX)
    summary = load_articles(FOLDER+"Generated_"+model, EVAL_INDEX)
    n = len(reference)
    results = {'precision': [], 'recall': [], 'f1': []}

    # ROUGE evaluation
    if metric=="rouge1" or metric=="rougeL":
        scorer = rouge_scorer.RougeScorer([metric], use_stemmer=True)
        for i in range(n):
            score = scorer.score(target=reference[i], prediction=summary[i])
            precision, recall, fmeasure = score[metric]
            results['precision'].append(precision)
            results['recall'].append(recall)
            results['f1'].append(fmeasure)
            
    # BertScore evaluation
    if metric=="BertScore":
        results = BERT.compute(predictions=summary, references=reference, lang="en")
        
    return results

In [7]:
def evaluate():
    for metric in METRICS:
        print("Metric = {}".format(metric))
        for model in MODELS:
            print("\tModel = {}".format(model))
            results = results = score(model=model, metric=metric)
            print("\t\tPrecision: {}".format(round(np.average(results['precision']), 3)))
            print("\t\tRecall: {}".format(round(np.average(results['recall']), 3)))
            print("\t\tF1: {}".format(round(np.average(results['f1']), 3)))

In [12]:
evaluate()

Metric = rouge1
	Model = TextRank


FileNotFoundError: ignored